## 1. GPU Checker

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

Wed Nov 23 02:25:54 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU')
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU


## 2. Image Files Preparation

In [1]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)	
import matplotlib.pyplot as plt
import os
os.chdir(r'D:\ME\GeorgeTown\FALL2022\ANLY-590\ANLY590\Data')
import numpy as np
import scipy.sparse
from tqdm import tqdm
import functools

from sklearn.utils import class_weight
import matplotlib.pyplot as plt


Bad key text.latex.preview in file C:\Users\Lovecraft.Qiu\AppData\Roaming\Python\Python37\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file C:\Users\Lovecraft.Qiu\AppData\Roaming\Python\Python37\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.3/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file C:\Users\Lovecraft.Qiu\AppData\Roaming\Python\Python37\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 418 ('savefig.jpeg_qualit

In [10]:
#The samples in test_csv will not be used for training & validation
train_csv=pd.read_csv('Data/train.csv')
train_csv.head()

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069


In [11]:
#The samples in test_csv will not be used during training
test_csv=pd.read_csv('Data/test.csv')
test_csv.head()

,posting_id,image,image_phash,title
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng


In [12]:
train_csv.label_group.value_counts().reset_index().rename(columns={'index':'label_group','label_group':'count'}).head()

,label_group,count
0,994676122,51
1,1163569239,51
2,1141798720,51
3,159351600,51
4,562358068,51


In [47]:
train_csv["image"] = "Data/train_images/"+train_csv["image"]
train_csv["label_group"] = train_csv["label_group"].astype("str")

In [48]:
test_ratio=0.1
label_count=train_csv.label_group.value_counts().reset_index().rename(columns={'index':'label_group','label_group':'count'})
sample_num={}

for i in range(label_count.shape[0]):
    num_to_sample=label_count['count'].iloc[i]*test_ratio
    if num_to_sample<2:
        continue
    sample_num[label_count['label_group'].iloc[i]]=int(num_to_sample)

sample_num

{'994676122': 5,
 '1163569239': 5,
 '1141798720': 5,
 '159351600': 5,
 '562358068': 5,
 '3113678103': 5,
 '3627744656': 5,
 '3206118280': 4,
 '1733221456': 4,
 '1166650192': 4,
 '1091404026': 4,
 '3489985175': 4,
 '1237550763': 3,
 '1746655739': 3,
 '452508504': 3,
 '731330024': 3,
 '4184214313': 3,
 '2008989859': 3,
 '1575763518': 3,
 '66247839': 3,
 '871679006': 3,
 '952183003': 3,
 '2259957740': 3,
 '3001123709': 3,
 '2999794436': 3,
 '2935813666': 3,
 '3926100920': 3,
 '656698835': 3,
 '1226500780': 3,
 '997220911': 3,
 '1201602115': 2,
 '821583868': 2,
 '3717044186': 2,
 '3868183614': 2,
 '1135976474': 2,
 '3326267479': 2,
 '3040690230': 2,
 '927285629': 2,
 '1306578136': 2,
 '2156459496': 2,
 '4141124289': 2,
 '2123332638': 2,
 '418991059': 2,
 '1569494229': 2,
 '3441184770': 2,
 '777596345': 2,
 '1744240905': 2,
 '2956941947': 2,
 '1065450055': 2,
 '1544174053': 2,
 '1088754866': 2,
 '2911646536': 2,
 '4277487223': 2,
 '3433277712': 2,
 '962477933': 2,
 '1285119273': 2,
 '403861

In [49]:
valid_idx=[list(np.where(train_csv['label_group']==label)[0][:sample_num[label]]) for label in sample_num]
valid_idx=[idx for group in valid_idx for idx in group]

train_idx=[i for i in range(train_csv.shape[0]) if i not in valid_idx]

print('TrainingSet Size:{}'.format(len(train_idx)))
print('ValidationSet Size:{}'.format(len(valid_idx)))

TrainingSet Size:34085
ValidationSet Size:165


In [50]:
train = train_csv.iloc[train_idx,:]
test = train_csv.iloc[valid_idx,:]

In [54]:
#train, val = train_test_split(train, test_size=0.3, stratify=train["label_group"], random_state=910)

## 3. Convert Images into Numeric Array

In [26]:
batch_size=64

In [27]:
# build image data generator
img_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=30, 
                                   width_shift_range=0.2,
                                   height_shift_range=0.2, 
                                   horizontal_flip = 'true')

In [31]:
train_generator=img_datagen.flow_from_dataframe(dataframe=train,
                                                x_col="image",
                                                y_col="label_group",
                                                batch_size=batch_size,
                                                seed=910,
                                                shuffle=False,
                                                target_size=(224, 224))

Found 34085 validated image filenames belonging to 11014 classes.


In [32]:
test_generator=img_datagen.flow_from_dataframe(dataframe=test,
                                               x_col="image",
                                                y_col="label_group",
                                               batch_size=batch_size,
                                               seed=3313,
                                               shuffle=False,
                                               target_size=(299, 299))

Found 165 validated image filenames belonging to 58 classes.


## 4. Define Common Model Parameters

In [81]:
steps_per_epoch = len(train)//batch_size
#validation_steps = len(test)//batch_size
N_label = len(set(train.label_group))
print(steps_per_epoch, N_label)

532 11014


In [82]:
 from keras import callbacks
 callback = callbacks.EarlyStopping(monitor='loss', patience=3, verbose=1, mode='auto', min_delta=0)

## 5. Model - InceptionV3

In [83]:
from sklearn.metrics import confusion_matrix, roc_auc_score, classification_report
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
import tensorflow as tf
from scipy import ndimage, misc
import skimage
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Sequential
from keras.layers.pooling import GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Dropout

In [84]:
i_model = InceptionV3(weights= 'imagenet', include_top=False, input_shape=(224, 224, 3))

In [85]:
for layer in i_model.layers:
    layer.trainable = False

In [86]:
def build_inceptionv3():
    model = Sequential()
    model.add(i_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dense(128))
    model.add(Dropout(0.2))
    model.add(Dense(N_label, activation = 'softmax'))
    #model.summary()
    model.compile(optimizer = Adam(),
                  loss="categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [87]:
model_inceptionv3 = build_inceptionv3()

In [88]:
model_inceptionv3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d_4   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_8 (Dense)             (None, 128)               262272    
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_9 (Dense)             (None, 11014)             1420806   
                                                                 
Total params: 23,485,862
Trainable params: 1,683,078
Non-trainable params: 21,802,784
__________________________________

In [89]:
with tf.device('/device:GPU:0'):
  history_inceptionv3 = model_inceptionv3.fit(train_generator, 
                                              steps_per_epoch = steps_per_epoch,
                                              epochs = 15, 
                                              callbacks=[callback],
                                              verbose = 2)

Epoch 1/15
532/532 - 612s - loss: 8.7661 - accuracy: 0.0251 - 612s/epoch - 1s/step
Epoch 2/15
532/532 - 588s - loss: 6.7325 - accuracy: 0.1144 - 588s/epoch - 1s/step
Epoch 3/15
532/532 - 585s - loss: 4.7136 - accuracy: 0.2440 - 585s/epoch - 1s/step
Epoch 4/15
532/532 - 581s - loss: 3.3918 - accuracy: 0.3760 - 581s/epoch - 1s/step
Epoch 5/15
532/532 - 581s - loss: 2.5958 - accuracy: 0.4830 - 581s/epoch - 1s/step
Epoch 6/15
532/532 - 588s - loss: 2.0733 - accuracy: 0.5669 - 588s/epoch - 1s/step
Epoch 7/15
532/532 - 583s - loss: 1.7254 - accuracy: 0.6267 - 583s/epoch - 1s/step
Epoch 8/15
532/532 - 586s - loss: 1.4858 - accuracy: 0.6684 - 586s/epoch - 1s/step
Epoch 9/15
532/532 - 581s - loss: 1.3396 - accuracy: 0.6963 - 581s/epoch - 1s/step
Epoch 10/15
532/532 - 582s - loss: 1.2123 - accuracy: 0.7223 - 582s/epoch - 1s/step
Epoch 11/15
532/532 - 580s - loss: 1.1251 - accuracy: 0.7390 - 580s/epoch - 1s/step
Epoch 12/15
532/532 - 579s - loss: 1.0482 - accuracy: 0.7552 - 579s/epoch - 1s/step
E

In [91]:
model_inceptionv3.save("inceptionv3")

In [92]:
X_train_prob_matrix = model_inceptionv3.predict(train_generator)

533/533 [==============================] - 594s 1s/step


In [94]:
np.save("X_train_prob_matrix", X_train_prob_matrix)

In [95]:
X_test_prob_matrix = model_inceptionv3.predict(test_generator)

3/3 [==============================] - 7s 3s/step


In [96]:
np.save("X_test_prob_matrix", X_test_prob_matrix)